In [1]:


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator

from sklearn.metrics import classification_report, confusion_matrix
print("TensorFlow version:", tf.__version__)

# -----------------------------
# 2) User settings (edit paths)
# -----------------------------
# Set this to where your dataset root is. Expect structure:
# DATA_DIR/
#   train/
#     tumour_class_1/
#     tumour_class_2/
#     ...
#   val/   OR you can let code split train->val automatically
#   test/
DATA_DIR = "/content/dataset"    # <<--- change this
IMG_SIZE = (224, 224)            # common size for transfer models
BATCH_SIZE = 16
SEED = 42
NUM_EPOCHS = 20
AUTOTUNE = tf.data.AUTOTUNE

# If your dataset is a single folder with subfolders for each class, great.
# If you have a CSV mapping filenames->labels, you'll need a small loader change.

# -----------------------------
# 3) Helper: list classes and counts
# -----------------------------
def print_dataset_summary(data_dir):
    print("Dataset summary for:", data_dir)
    for split in ["train", "val", "test"]:
        split_dir = Path(data_dir) / split
        if split_dir.exists():
            classes = [p.name for p in split_dir.iterdir() if p.is_dir()]
            print(f"  {split} classes:", classes)
            for cls in classes:
                n = len(list((split_dir/cls).glob("*")))
                print(f"    {cls}: {n}")
        else:
            print(f"  {split} not found (will auto-split train if needed)")

print_dataset_summary(DATA_DIR)

# -----------------------------
# 4) Prepare data generators (robust and simple)
# -----------------------------
# If no val folder, we'll create a train/val split from train.
train_dir = Path(DATA_DIR) / "train"
val_dir   = Path(DATA_DIR) / "val"
test_dir  = Path(DATA_DIR) / "test"

if not train_dir.exists():
    raise FileNotFoundError(f"Train folder not found at {train_dir}. Please set DATA_DIR correctly.")

# If val doesn't exist, we'll split the training set:
if not val_dir.exists():
    print("No 'val' folder — creating a validation split from train.")
    # We'll use ImageDataGenerator.flow_from_directory 'subset' functionality below.

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.08,
    height_shift_range=0.08,
    shear_range=0.08,
    zoom_range=0.08,
    horizontal_flip=True,
    vertical_flip=False,
    fill_mode='nearest',
    validation_split=0.2  # only used if val directory not present
)

test_datagen = ImageDataGenerator(rescale=1./255)

# Choose source dirs and create generators (prefer explicit 'val' directory)
if val_dir.exists():
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        seed=SEED,
        shuffle=True
    )
    validation_generator = test_datagen.flow_from_directory(
        val_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        seed=SEED,
        shuffle=False
    )
else:
    train_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        seed=SEED,
        shuffle=True,
        subset='training'
    )
    validation_generator = train_datagen.flow_from_directory(
        train_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        seed=SEED,
        shuffle=False,
        subset='validation'
    )

if test_dir.exists():
    test_generator = test_datagen.flow_from_directory(
        test_dir,
        target_size=IMG_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical',
        shuffle=False
    )
else:
    test_generator = None
    print("No test folder found — evaluation will use validation set.")

CLASS_NAMES = list(train_generator.class_indices.keys())
NUM_CLASSES = len(CLASS_NAMES)
print("Classes:", CLASS_NAMES)

# -----------------------------
# 5) Build model options
# -----------------------------
def build_small_cnn(input_shape=(*IMG_SIZE, 3), num_classes=NUM_CLASSES):
    """
    Simple descriptive CNN from scratch for learning & demonstration.
    Not necessarily SOTA for brain MRI, but useful for learning and baseline.
    """
    inputs = keras.Input(shape=input_shape)
    x = layers.Conv2D(32, 3, padding='same', activation='relu')(inputs)
    x = layers.MaxPooling2D(2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(64, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv2D(128, 3, padding='same', activation='relu')(x)
    x = layers.MaxPooling2D(2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(128, activation='relu')(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs, name="small_cnn")
    return model

def build_transfer_model(input_shape=(*IMG_SIZE, 3), num_classes=NUM_CLASSES, base_model_name="EfficientNetB0"):
    """
    Transfer learning version using EfficientNetB0 (available in TF Keras).
    We freeze the base, add a small head, then fine-tune later if desired.
    """
    if base_model_name == "EfficientNetB0":
        base = tf.keras.applications.EfficientNetB0(
            include_top=False, weights='imagenet', input_shape=input_shape, pooling=None
        )
        preprocess = tf.keras.applications.efficientnet.preprocess_input
    elif base_model_name == "MobileNetV2":
        base = tf.keras.applications.MobileNetV2(include_top=False, weights='imagenet', input_shape=input_shape)
        preprocess = tf.keras.applications.mobilenet_v2.preprocess_input
    else:
        raise ValueError("Unsupported base model name")

    base.trainable = False  # freeze base initially

    inputs = keras.Input(shape=input_shape)
    x = preprocess(inputs)  # use corresponding preprocess
    x = base(x, training=False)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu')(x)
    x = layers.Dropout(0.25)(x)
    outputs = layers.Dense(num_classes, activation='softmax')(x)

    model = keras.Model(inputs, outputs, name=f"transfer_{base_model_name}")
    return model

# -----------------------------
# 6) Choose model and compile
# -----------------------------
USE_TRANSFER = True  # set False to use small CNN from scratch

if USE_TRANSFER:
    model = build_transfer_model()
else:
    model = build_small_cnn()

model.summary()

model.compile(
    optimizer=keras.optimizers.Adam(learning_rate=1e-4),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

# -----------------------------
# 7) Callbacks
# -----------------------------
checkpoint_path = "best_model.h5"
callbacks = [
    keras.callbacks.ModelCheckpoint(checkpoint_path, monitor='val_accuracy', save_best_only=True, verbose=1),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1),
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True, verbose=1)
]

# -----------------------------
# 8) Train
# -----------------------------
history = model.fit(
    train_generator,
    epochs=NUM_EPOCHS,
    validation_data=validation_generator,
    callbacks=callbacks
)

# -----------------------------
# 9) Training curves
# -----------------------------
def plot_history(h):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(h.history['loss'], label='train_loss')
    plt.plot(h.history['val_loss'], label='val_loss')
    plt.legend(); plt.title('Loss')

    plt.subplot(1,2,2)
    plt.plot(h.history['accuracy'], label='train_acc')
    plt.plot(h.history['val_accuracy'], label='val_acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

plot_history(history)

# -----------------------------
# 10) Evaluate on test (or val) and show confusion matrix
# -----------------------------
if test_generator:
    eval_gen = test_generator
else:
    eval_gen = validation_generator

# Predict labels
y_pred_probs = model.predict(eval_gen, verbose=1)
y_pred = np.argmax(y_pred_probs, axis=1)
y_true = eval_gen.classes  # note: flow_from_directory sets .classes
labels = CLASS_NAMES

# Classification report
print("Classification Report:")
print(classification_report(y_true, y_pred, target_names=labels, digits=4))

# Confusion matrix
cm = confusion_matrix(y_true, y_pred)
def plot_confusion_matrix(cm, classes):
    plt.figure(figsize=(6,6))
    plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    plt.title('Confusion matrix')
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)
    fmt = 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")
    plt.ylabel('True label'); plt.xlabel('Predicted label')
    plt.tight_layout()
    plt.show()

plot_confusion_matrix(cm, labels)

# -----------------------------
# 11) Save final model (already checkpointed best weights)
# -----------------------------
model.save("brain_tumor_model_final.h5")
print("Saved model to brain_tumor_model_final.h5")

# -----------------------------
# 12) Grad-CAM (explainability)
# -----------------------------
# We'll implement Grad-CAM for Keras models to visualize which image regions influenced a prediction.
# This implementation assumes the final conv layer is accessible by name; we find the last conv layer automatically.

def get_last_conv_layer(model):
    # find the last convolutional layer in the model
    for layer in reversed(model.layers):
        if isinstance(layer, layers.Conv2D) or 'conv' in layer.name:
            return layer.name
    raise ValueError("Could not find a convolutional layer in the model")

last_conv_layer_name = get_last_conv_layer(model)
print("Last conv layer:", last_conv_layer_name)

# Grad-CAM function
def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model(
        [model.inputs], [model.get_layer(last_conv_layer_name).output, model.output]
    )
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]

    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

# Display sample Grad-CAMs from validation set
def display_gradcam_from_generator(generator, model, n=4):
    x_batch, y_batch = next(generator)
    plt.figure(figsize=(12, 3*n))
    for i in range(n):
        img = x_batch[i]
        img_input = np.expand_dims(img, axis=0)
        preds = model.predict(img_input)
        pred_class = np.argmax(preds[0])
        heatmap = make_gradcam_heatmap(img_input, model, last_conv_layer_name, pred_index=pred_class)
        # Resize heatmap to image size
        heatmap = cv2.resize(heatmap, (IMG_SIZE[1], IMG_SIZE[0]))
        heatmap = np.uint8(255 * heatmap)
        heatmap_color = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
        superimposed = heatmap_color * 0.4 + np.uint8(255 * img)
        plt.subplot(n, 3, i*3 + 1)
        plt.imshow(img); plt.title(f"True: {CLASS_NAMES[np.argmax(y_batch[i])]}")
        plt.axis('off')
        plt.subplot(n, 3, i*3 + 2)
        plt.imshow(heatmap, cmap='jet'); plt.title("Heatmap"); plt.axis('off')
        plt.subplot(n, 3, i*3 + 3)
        plt.imshow(np.uint8(superimposed)); plt.title(f"Pred: {CLASS_NAMES[pred_class]}")
        plt.axis('off')
    plt.tight_layout()
    plt.show()

# Run Grad-CAM visualization (uses validation generator so it won't shuffle)
display_gradcam_from_generator(validation_generator, model, n=4)

# -----------------------------
# 13) Optional: Fine-tune (if using transfer learning)
# -----------------------------
if USE_TRANSFER:
    # Unfreeze some of the base model and fine-tune at a lower lr
    base_model = [l for l in model.layers if 'efficientnet' in l.name or 'mobilenet' in l.name]
    if base_model:
        print("Attempting fine-tuning: unfreezing last layers of the base model.")
        # Unfreeze entire base for simplicity (or selectively unfreeze)
        for layer in base_model[0].layers:
            layer.trainable = True
        model.compile(
            optimizer=keras.optimizers.Adam(learning_rate=1e-5),
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        fine_tune_epochs = 10
        ft_history = model.fit(
            train_generator,
            epochs=fine_tune_epochs,
            validation_data=validation_generator,
            callbacks=[
                keras.callbacks.ModelCheckpoint("best_model_finetuned.h5", monitor='val_accuracy', save_best_only=True),
                keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3)
            ]
        )
        print("Fine-tuning finished; best weights saved to best_model_finetuned.h5")
    else:
        print("Could not find a known base model to unfreeze for fine-tuning.")

# End of notebook


ModuleNotFoundError: No module named 'tensorflow'

# 🧠 Brain Tumor Detection using CNN  
This project detects **brain tumors from MRI images** using a **Convolutional Neural Network (CNN)** built with TensorFlow / Keras.  

### **Goals**
- Classify MRI scans as tumor types or normal.
- Use **data augmentation** for better generalization.
- Optionally use **transfer learning** (EfficientNet).
- Visualize attention areas using **Grad-CAM**.
- Save model for deployment on a web app.

---



### -----------------------------
### 1) Install / import packages
### -----------------------------
###  In Colab uncomment the pip lines below if libs are missing:
### pip install -q tensorflow==2.10.0 scikit-learn matplotlib opencv-python seaborn

In [6]:
import os, random, numpy as np, matplotlib.pyplot as plt, cv2, itertools
from pathlib import Path
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report, confusion_matrix

AttributeError: partially initialized module 'pandas' from 'C:\Users\prem\AppData\Roaming\Python\Python313\site-packages\pandas\__init__.py' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from tensorflow.keras.optimizers import Adam

# Enable reproducible results
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# For cleaner plotting
plt.style.use('seaborn')

AttributeError: partially initialized module 'pandas' from 'C:\Users\prem\AppData\Roaming\Python\Python313\site-packages\pandas\__init__.py' has no attribute '_pandas_datetime_CAPI' (most likely due to a circular import)

## 1️⃣ Dataset Setup

We assume a directory structure like:

dataset/
├── train/
│ ├── glioma/
│ ├── meningioma/
│ ├── pituitary/
│ └── no_tumor/
├── val/ (optional)
└── test/ (optional)

In [9]:
train_path = '/content/brain_tumor_dataset/Training'
test_path = '/content/brain_tumor_dataset/Testing'

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    zoom_range=0.2,
    horizontal_flip=True
)

test_datagen = ImageDataGenerator(rescale=1./255)

train_gen = train_datagen.flow_from_directory(
    train_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_gen = test_datagen.flow_from_directory(
    test_path,
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


FileNotFoundError: [Errno 2] No such file or directory: '/content/brain_tumor_dataset/Training'

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')  # 4 classes
])

model.summary()


In [ ]:
model.compile(
    optimizer=Adam(learning_rate=0.001),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)


In [ ]:
history = model.fit(
    train_gen,
    validation_data=test_gen,
    epochs=10,
    verbose=1
)


In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.legend()

plt.show()


In [ ]:
model.save('brain_tumor_model_final.h5')
print("✅ Model saved successfully as brain_tumor_model_final.h5")
